In [2]:
from PIL import Image
import base64
import pandas as pd
import numpy as np
import json
import io 

from IPython.display import HTML



# Load data
df1 = pd.read_csv('EDA.csv')
df2 = pd.read_csv('TEMP.csv')

# Assuming 'Time' is in datetime format
df1['Formatted_Date'] = pd.to_datetime(df1['1697776068.000000'], unit='s')
time_increment = 250
df1['Formatted_Date'] += pd.to_timedelta(df1.index * time_increment, unit='ms')

# Extract relevant columns
data = pd.DataFrame({
    'Time': df1['Formatted_Date'],
    'EDA': df1['1697776068.000000'],
    'Temp': df2['1697776068.000000']
})

time=df1['Formatted_Date']
eda = df1['1697776068.000000']
temp = df2['1697776068.000000']
time = pd.to_datetime(time)

# Create a DataFrame from the arrays
data = pd.DataFrame({'Time': time, 'EDA': eda, 'Temp': temp})

# Sort the data by the 'Time' column (if not already sorted)
data = data.sort_values('Time')

# Apply moving average to 'EDA' and 'Temp' data
window_size = 10  # Adjust the window size for smoothing
data['Smoothed_EDA'] = data['EDA'].rolling(window=window_size).mean()
data['Smoothed_Temp'] = data['Temp'].rolling(window=window_size).mean()

# Define a mapping function to map 'Temp' values to the specified ranges
def map_temp(temp):
    if temp < 28:
        return 1
    elif 29 <= temp < 30:
        return 2
    elif 30 <= temp < 31:
        return 3
    elif 31 <= temp < 32:
        return 4
    elif 32 <= temp < 33:
        return 5
    elif 33 <= temp < 34:
        return 6
    elif 34 <= temp < 35:
        return 7
    elif 35 <= temp < 36:
        return 8
    elif 36 <= temp < 37:
        return 9
    else:
        return 10
    

data['Modified Temp'] = data['Temp'].apply(map_temp)

def map_eda(eda):
    return (eda / 13) * 10

# Apply the scaling function to the 'EDA' column
data['Modified EDA'] = data['EDA'].apply(map_eda)

# Calculate the average of 'EDA' and 'Temp'
average_eda = data['Modified EDA'].mean()
average_temp = data['Modified Temp'].mean()

# Assuming 'Time' is in datetime format
data['Time'] = pd.to_datetime(data['Time'])

# Extract the hour from the 'Time' column
data['Hour'] = data['Time'].dt.hour

# Categorize time into morning, afternoon, and night based on the hour
bins = [-1, 5, 12, 18]
labels = ['Night', 'Morning', 'Afternoon']  # Adjusted labels to ensure uniqueness
data['Time_Category'] = pd.cut(data['Hour'], bins=bins, labels=labels, include_lowest=True)

# Calculate the average for 'Modified EDA' and 'Modified Temp' for each time category
average_by_time = data.groupby('Time_Category').agg({'Modified EDA': 'mean', 'Modified Temp': 'mean'}).reset_index()

# Display the calculated averages
print(average_by_time)
def above_mean_matrix(average_by_time, data):
    # Calculate the mean of 'Modified EDA' and 'Modified Temp' for the entire day
    overall_mean_eda = data['Modified EDA'].mean()
    overall_mean_temp = data['Modified Temp'].mean()

    # Create a matrix of "Yes" or "No" based on whether the values are above the mean
    matrix = pd.DataFrame(index=average_by_time['Time_Category'], columns=['Above Mean EDA', 'Above Mean Temp'])

    matrix['Above Mean EDA'] = np.where(average_by_time['Modified EDA'] > overall_mean_eda, 'Yes', 'No')
    matrix['Above Mean Temp'] = np.where(average_by_time['Modified Temp'] > overall_mean_temp, 'Yes', 'No')

    return matrix

average_by_time = data.groupby('Time_Category').agg({'Modified EDA': 'mean', 'Modified Temp': 'mean'}).reset_index()

# Example usage:
matrix_result = above_mean_matrix(average_by_time, data)

# Assuming you have loaded the images
very_stressed_image = '\\10 weeks Challenge\\Very_stressed.jpg'
not_stressed_image = '\\10 weeks Challenge\\Not_stressed.jpg'
stressed_image = '\\10 weeks Challenge\\Stressed.jpg'
maybe_stressed_image = '\\10 weeks Challenge\\Maybe_stressed.jpg'

def image_to_base64(image):
    buffered = io.BytesIO()
    image.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode('utf-8')

def combine_stress_levels_with_images(matrix_result):
    stress_levels = []

    for index, row in matrix_result.iterrows():
        if row['Above Mean EDA'] == 'Yes' and row['Above Mean Temp'] == 'Yes':
            stress_levels.append(very_stressed_image)
        elif row['Above Mean EDA'] == 'No' and row['Above Mean Temp'] == 'No':
            stress_levels.append(not_stressed_image)
        elif row['Above Mean EDA'] == 'Yes' and row['Above Mean Temp'] == 'No':
            stress_levels.append(stressed_image)
        elif row['Above Mean EDA'] == 'No' and row['Above Mean Temp'] == 'Yes':
            stress_levels.append(maybe_stressed_image)

    stress_level_result = pd.DataFrame({'Time_Category': matrix_result.index, 'Stress_Level': stress_levels})

    # Convert images to base64 and embed them in HTML
    stress_level_result['Stress_Level'] = stress_level_result['Stress_Level'].apply(lambda img: f'<img src="data:image/png;base64,{image_to_base64(img)}" width="50">')

    # Reorder the rows based on the desired order: Morning, Afternoon, Night
    stress_level_result = stress_level_result.reindex([1, 2, 0])

    return stress_level_result

# Example usage:
stress_level_result_with_images = combine_stress_levels_with_images(matrix_result)
display(HTML(stress_level_result_with_images.to_html(escape=False)))



  Time_Category  Modified EDA  Modified Temp
0         Night      2.780073       5.714785
1       Morning      4.558853       6.001378
2     Afternoon           NaN            NaN


C:\Users\elpid\AppData\Local\Temp\ipykernel_36532\731261887.py:92: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  average_by_time = data.groupby('Time_Category').agg({'Modified EDA': 'mean', 'Modified Temp': 'mean'}).reset_index()
C:\Users\elpid\AppData\Local\Temp\ipykernel_36532\731261887.py:109: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  average_by_time = data.groupby('Time_Category').agg({'Modified EDA': 'mean', 'Modified Temp': 'mean'}).reset_index()


NameError: name 'not_stressed_image' is not defined